In [73]:
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt
from ipywidgets import interactive, fixed
plt.rcParams['figure.figsize'] = [10, 9]


In [35]:
def circleRadius(b, c, d):
    """
    Regner ut radius basert på tre punkt 
    https://stackoverflow.com/questions/52990094/calculate-circle-given-3-points-code-explanation
    """
    temp = c[0]**2 + c[1]**2
    bc = (b[0]**2 + b[1]**2 - temp) / 2
    cd = (temp - d[0]**2 - d[1]**2) / 2
    det = (b[0] - c[0]) * (c[1] - d[1]) - (c[0] - d[0]) * (b[1] - c[1])

    if abs(det) < 1.0e-10:
        return None

    # Center of circle
    cx = (bc*(c[1] - d[1]) - cd*(b[1] - c[1])) / det
    cy = ((b[0] - c[0]) * cd - (c[0] - d[0]) * bc) / det

    radius = ((cx - b[0])**2 + (cy - b[1])**2)**.5

    return radius

# Første forsøk: Fra ytre til indre sving

Her blir det en diskontiuitet: Fra å kjøre rett frem (langs ytre vegkant) så er man plutselig i en sving som starter der vegkanten begynner å krumme. En perfekt svinglinje starter et lite stykke _*før*_ svingen, der sirkelbuen tangerer vegkanten.  

In [140]:
def sirkler( IndreKrumningsRadius = 60, Vegbredde = 3, theta=45, Bilbredde = 2  ):
    Ri = IndreKrumningsRadius
        
    n_samples = 500
#     angle = np.linspace( -theta * np.pi / 360, theta * np.pi / 360, n_samples )
    angle = np.linspace( 0, theta * np.pi / 360, n_samples )
    
    # Regner ut sirkelbue høyre og venstre vegkant (alternativt ene vegkant + senterlinje)
    x1 = Ri * np.cos( angle )
    y1 = Ri * np.sin( angle )
    x2 = (Ri + Vegbredde) * np.cos( angle )
    y2 = (Ri + Vegbredde) * np.sin( angle )
     
    # Plotter sirkelbue vegkanter
    fix, ax = plt.subplots( 1 )
    ax.plot( x1, y1, '-', color='dimgrey', linewidth=3)
    ax.plot( x2, y2, '-', color='dimgrey', linewidth=3)

    # Regner ut berøringspunkt ytre vegkant - indre vegkant - ytre vegkant, minus bilbredde
    bb = Bilbredde / 2 
    p1 = [ (Ri+Vegbredde-bb) * np.cos( theta * np.pi / 360), (Ri+Vegbredde-bb) * np.sin( theta * np.pi / 360) ]
    p2 = [ Ri+bb, 0 ]
    p3 = [ (Ri+Vegbredde-bb) * np.cos( theta * np.pi / 360), -(Ri+Vegbredde-bb) * np.sin( theta * np.pi / 360) ]
    
    # Sirkelradius 
    storR = circleRadius( p1, p2, p3)
    
    # Beregner åpningsvinkel
    theta2 = np.arcsin( p1[1] / storR )
    
    # Nullpunkt for den store sirkelen på X-aksen
    forskyving = Ri + bb - storR 
    
#     angle2 = np.linspace( -theta2, theta2, n_samples  )
    angle2 = np.linspace( 0, theta2, n_samples  )
    
    x3 = storR * np.cos( angle2 ) + forskyving 
    y3 = storR * np.sin( angle2 ) 
    
    if p2[0] > p1[0]: 
        ax.plot( x3, y3, '-r', linewidth=1)
        perfektLengde = storR * 2 * theta2 
    else: 
        ax.plot( [ p1[0], p3[0] ], [p1[1], p3[1] ], '-r', linewidth=1 )
        storR = 'uendelig (rak linje)'
        perfektLengde = 2 * p1[1]
    
    ax.plot( p1[0], p1[1], 'om')
    ax.plot( p2[0], p2[1], 'om')
#     ax.plot( p3[0], p3[1], 'om')
    
    print( 'Krumningsradius perfekt bane:', storR )
    print( 'Lengde perfekt bane:', '\t',  round( perfektLengde, 2) )
    print( 'Lengde indre vegkant:', '\t', round(Ri * theta * np.pi / 180, 2) )
    
    # Plotter bil: 
    xbil = [Ri, Ri+Bilbredde, Ri+Bilbredde, Ri,  Ri ]
    ybil = [-2.5, -2.5,       2.5,          2.5, -2.5 ] 
    xpil = [Ri,              Ri+Bilbredde,    Ri+Bilbredde/2, Ri ]
    ypil = [2.5-Bilbredde/2, 2.5-Bilbredde/2, 2.5,            2.5-Bilbredde/2 ]
    if Bilbredde > 0:
        ax.plot( xbil, ybil, '-', color='tomato')
    if Bilbredde > 1:
        ax.plot( xpil, ypil, '-', color='tomato')

    # Plotter vinkelrett på linja Origo - slutt på sirkelbuen 
    slope = -1 / ( y1[-1] / x1[-1] )
    # Kjenner et punkt (x1, y1) på linja og kan bruke formelen y = slope * (x - x1) + y1 
    Xrak = [ x1[-1], 0.9*x1[-1] ]
    Yrak = slope * ( Xrak - x1[-1] ) + y1[-1] 
    ax.plot( Xrak, Yrak, '-', color = 'y', linewidth = 3)
    
    Xrak2 = [ x2[-1], 0.9*x2[-1]]
    Yrak2 = slope * ( Xrak2 - x2[-1]) + y2[-1]
    ax.plot( Xrak2, Yrak2, '-', color='y', linewidth = 3)
    
    ## Beregner enhetsvektor parallent med høyre vegkant rettlinje
    # Lengden ... 
    rak_L = np.sqrt( np.square(Xrak2[1]-Xrak2[0]) + np.square(Yrak2[1]-Yrak2[0])  )
    # Vektor p1->p2 = vektor p1->Origo + vektor fra Origo->p2 = -1*Op1 + Op2
    # = <x2-x1, y2-y1>
    # Og skalerer slik at vi har enhetsvektor
    vegkantVektor = np.array( [ (Xrak2[1]-Xrak2[0])/rak_L, (Yrak2[1]-Yrak2[0])/rak_L ] )

    # Enhetsvektor for y-aksen 
    yAkseVektor = np.array( [0, -1 ] )
    
    # Plott disse vektorene
    plottepunkt = Ri-5
    ax.arrow( plottepunkt, 0, yAkseVektor[0], yAkseVektor[1]   )
    ax.arrow( plottepunkt, 0, vegkantVektor[0], vegkantVektor[1]   )
    
    # dot_product = np.dot( yAkseVektor, vegkantVektor)
    # vinkel = np.arccos( dot_product)
    vinkel = np.arccos( np.dot( yAkseVektor, vegkantVektor) )
    print( "Vinkel:", np.degrees( vinkel ))
    
    # Finne krysningspunkt mellom rett høyre vegkant og tangent ved Apex (Ri, 0)
    # https://stackoverflow.com/questions/3252194/numpy-and-line-intersections
    
    
    ax.set_aspect( 1) 
    plt.show()
    

In [144]:
interactive( sirkler, IndreKrumningsradius=(20, 200), Vegbredde=(1,9), theta=(5, 90), Bilbredde=(0,2))

interactive(children=(IntSlider(value=60, description='IndreKrumningsRadius', max=180, min=-60), IntSlider(val…

Men dette er ingen god løsning på problemet vårt - det er en diskontinuitet (uendelig krapp) overgang fra å kjøre rett fram til den pene røde svingebuen. 

Det problemet vi _*egentlig*_ skal løse kan foruleres slik: Finne sirkel med tre tangenter, definert ved 
  * høyre vegkant før svingen
  * linja `x = IndreKrumningsRadius` 
  * høyre kant etter svingen
    
https://math.stackexchange.com/questions/656497/find-a-circles-radius-with-three-known-tangent-lines

Ett steg på veien: Plott den rette vegen, dvs tangenten til de to vegkant-i-svingen sirklene våre fra start/slutt på svingen og rett fram vekk fra svingen: https://www.mathworks.com/matlabcentral/answers/258780-how-to-plot-tangent-line-from-specified-point-to-a-circle

Å finne sirkelens sentrum kan gjøres ved å trekke linja definert ved halve vinkelåpningen mellom parvise tangentlinjer. Den ene slike linja har vi, det er halve åpningsvinkelen mellom høyre vegkant før og etter svingen, og er rett og slett X-aksen (`y = 0 `). Så vi må finne linja for halve åpningsvinkelen mellom linja `y = IndreKrumningsRadius` og den ene høyre-vegkant-rettlinja. Skjæringspunktet med X-aksen er senter for sirkelen.   

In [173]:
def get_intersect(a1, a2, b1, b2):
    """ 
    Returns the point of intersection of the lines passing through a2,a1 and b2,b1.
    a1: [x, y] a point on the first line
    a2: [x, y] another point on the first line
    b1: [x, y] a point on the second line
    b2: [x, y] another point on the second line
    
    https://stackoverflow.com/a/42727584
    """
    s = np.vstack([a1,a2,b1,b2])        # s for stacked
    h = np.hstack((s, np.ones((4, 1)))) # h for homogeneous
    l1 = np.cross(h[0], h[1])           # get first line
    l2 = np.cross(h[2], h[3])           # get second line
    x, y, z = np.cross(l1, l2)          # point of intersection
    if z == 0:                          # lines are parallel
        return (float('inf'), float('inf'))
    return (x/z, y/z)

In [175]:
get_intersect( np.array( [0, 0]), np.array([1,1,]), np.array([6, 0]), np.array([3, 3]))

(3.0, 3.0)

In [285]:
def sirkle2( IndreKrumningsRadius = 60, Vegbredde = 3, theta=45, Bilbredde = 2  ):
    Ri = IndreKrumningsRadius
        
    n_samples = 500
#     angle = np.linspace( -theta * np.pi / 360, theta * np.pi / 360, n_samples )
    angle = np.linspace( 0, theta * np.pi / 360, n_samples )
    
    # Regner ut sirkelbue høyre og venstre vegkant (alternativt ene vegkant + senterlinje)
    x1 = Ri * np.cos( angle )
    y1 = Ri * np.sin( angle )
    x2 = (Ri + Vegbredde) * np.cos( angle )
    y2 = (Ri + Vegbredde) * np.sin( angle )
     
    # Plotter sirkelbue vegkanter
    fix, ax = plt.subplots( 1 )
    ax.plot( x1, y1, '-', color='dimgrey', linewidth=3)
    ax.plot( x2, y2, '-', color='dimgrey', linewidth=3)

    
#     # Plotter bil: 
#     xbil = [Ri, Ri+Bilbredde, Ri+Bilbredde, Ri,  Ri ]
#     ybil = [-2.5, -2.5,       2.5,          2.5, -2.5 ] 
#     xpil = [Ri,              Ri+Bilbredde,    Ri+Bilbredde/2, Ri ]
#     ypil = [2.5-Bilbredde/2, 2.5-Bilbredde/2, 2.5,            2.5-Bilbredde/2 ]
#     if Bilbredde > 0:
#         ax.plot( xbil, ybil, '-', color='tomato')
#     if Bilbredde > 1:
#         ax.plot( xpil, ypil, '-', color='tomato')

    # Finner stigningstallet for linja vinkelrett på linja Origo - slutt på sirkelbuen 
    slope = -1 / ( y1[-1] / x1[-1] )
    # Kjenner et punkt (x1, y1) på linja og kan bruke formelen y = slope * (x - x1) + y1 
    Xrak = [ x1[-1], 0.9*x1[-1] ]
    Yrak = slope * ( Xrak - x1[-1] ) + y1[-1] 
    ax.plot( Xrak, Yrak, '-', color = 'y', linewidth = 3)
    
    Xrak2 = [ x2[-1], 0.9*x2[-1]]
    Yrak2 = slope * ( Xrak2 - x2[-1]) + y2[-1]
    ax.plot( Xrak2, Yrak2, '-', color='r', linewidth = 3)
    
    ## Beregner enhetsvektor parallent med høyre vegkant rettlinje
    # Lengden ... 
    rak_L = np.sqrt( np.square(Xrak2[1]-Xrak2[0]) + np.square(Yrak2[1]-Yrak2[0])  )
    # Vektor p1->p2 = vektor p1->Origo + vektor fra Origo->p2 = -1*Op1 + Op2
    # = <x2-x1, y2-y1>
    # Og skalerer slik at vi har enhetsvektor
    vegkantVektor = np.array( [ (Xrak2[1]-Xrak2[0])/rak_L, (Yrak2[1]-Yrak2[0])/rak_L ] )

    # Enhetsvektor for y-aksen 
    yAkseVektor = np.array( [0, -1 ] )
    
#     # Plott disse vektorene
#     plottepunkt = Ri-5
#     ax.arrow( plottepunkt, 0, yAkseVektor[0], yAkseVektor[1]   )
#     ax.arrow( plottepunkt, 0, vegkantVektor[0], vegkantVektor[1]   )
    
    # dot_product = np.dot( yAkseVektor, vegkantVektor)
    # vinkel = np.arccos( dot_product)
    vinkel = np.arccos( np.dot( yAkseVektor, vegkantVektor) )
    print( "Vinkel:", np.degrees( vinkel ))
    
    # Finne krysningspunkt mellom rett høyre vegkant og tangent ved Apex (Ri, 0)
    # https://stackoverflow.com/questions/3252194/numpy-and-line-intersections
    Ptangentkryss = get_intersect( [Ri,       0 ],       [Ri,       10], 
                                   [Xrak2[1], Yrak2[1]], [Xrak2[0], Yrak2[0]])
    ax.plot( [Ri,  Ptangentkryss[0]], [0, Ptangentkryss[1]], '-r')
    print( 'Ptangentkryssing', Ptangentkryss)

    # Finn halvvinkel mellom rak veg og X-akse
    # Adder vinkelen til den rake vegen 
    halvVinkel = vinkel / 2 + np.arctan( slope ) + np.pi
    
    print( "Halvvinkel", np.degrees( halvVinkel))
    # Et punkt på linja 
    P1halvVinkel = [ Ptangentkryss[0] + 15 * np.cos( halvVinkel ), Ptangentkryss[1] + 15 * np.sin( halvVinkel )   ]
    ax.plot( [ Ptangentkryss[0], P1halvVinkel[0]], [ Ptangentkryss[1], P1halvVinkel[1] ], '--k'   )
    
    # Finn krysningspunkt mellom X-akse og halvvinkel 
    sentrum = get_intersect( Ptangentkryss, P1halvVinkel, (0, 0), (10, 0) )
#     ax.plot( [ Ptangentkryss[0], sentrum[0]], [ Ptangentkryss[1], sentrum[1]], '--r'  )
    
    # Sirkelbue for perfekt bane
    # Vinkel fra sirkelens midtpunkt til der sirkelen tangerer ytre vegbane 
    vinkel2 = halvVinkel - np.pi
    perfektRadius = (Ri +  abs( sentrum[0]) )
    anglePerfect = np.linspace(0, 2 *vinkel2 , n_samples )
    xPerfekt = perfektRadius *  np.cos( anglePerfect ) + sentrum[0] 
    yPerfekt = perfektRadius *  np.sin( anglePerfect ) 
    ax.plot( xPerfekt, yPerfekt, '-b')
    print( 'Radius:', perfektRadius )
    ax.plot( xPerfekt[-1], yPerfekt[-1], '-m')
    

    
    ax.set_aspect( 1) 
    plt.show()

In [253]:
np.degrees( np.pi)

180.0

In [286]:
interactive( sirkle2, IndreKrumningsradius=(30, 300), Vegbredde=(1,9), theta=(5, 90), Bilbredde=(0,2))

interactive(children=(IntSlider(value=60, description='IndreKrumningsRadius', max=180, min=-60), IntSlider(val…